In [0]:
#Write how many rows do you want (min. 250 000).
User_input = int(input("How many rows do you want? "))

How many rows do you want?  3000000

In [0]:
#importing needed libraries and their classses
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

import pandas as pd
from faker import Faker

from sqlalchemy import create_engine
from time import time
import datetime
import random
import uuid

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()
fake = Faker()

In [0]:

%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   983  100   983    0     0  40958      0 --:--:-- --:--:-- --:--:-- 40958
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    79  100    79    0     0   2078      0 --:--:-- --:--:-- --:--:--  2078
Hit:1 https://packages.microsoft.com/ubuntu/16.04/prod xenial InRelease
Hit:2 https://repos.azul.com/zulu/deb stable InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backport

In [0]:
def collect_attributes(input_df, filtered_column):
    
    output = []

    selected_values = input_df.select(filtered_column)
    collect_selected_values = selected_values.collect()

    
    for x in collect_selected_values:
        output.append(x[0])
        
    return output

In [0]:
# Define the connection details
prod_server_name = 'cat-demo-aero.database.windows.net'
prod_port = '1433'
prod_database_name = 'aero'
prod_user_name = 'sqladminuser'
prod_password = 'ThisIsOneGoodDemo8!'

# SQLAlchemy connection URL for SQL Server
prod_connection_url = f"mssql+pyodbc://{prod_user_name}:{prod_password}@{prod_server_name}:{prod_port}/{prod_database_name}?driver=ODBC+Driver+17+for+SQL+Server"

# Create the SQLAlchemy engine
prod_engine = create_engine(prod_connection_url)
   

In [0]:

# Read relevant data from Azure SQL Database into a DataFrame
table_name = 'COLUMNS'
table_schema = 'INFORMATION_SCHEMA'
table_fullname = table_schema + "." + table_name

information_schema_df = (spark.read
            .format("jdbc")
            .option("url", f"jdbc:sqlserver://{prod_server_name}:{prod_port};database={prod_database_name}")
            .option("user", prod_user_name)
            .option("password", prod_password)
            .option("dbtable", table_fullname)
            .load())

information_schema_filtered_df = information_schema_df.filter((information_schema_df.COLUMN_NAME.isin(['DESTINATION_ID', 'PLANE_ID', 'AIRLINE_ID', 'GATE_ID', 'LUGGAGE_TYPE_ID', 'TICKET_TYPE_ID']))).select("TABLE_SCHEMA", "TABLE_NAME", "COLUMN_NAME").orderBy("TABLE_NAME")

collected_information_schema = information_schema_filtered_df.collect()

# declare dictionary to store params in
parameters = {"AIRLINES_PARAMS":[],"DESTINATIONS_PARAMS":[],"GATES_PARAMS":[],"PLANES_PARAMS":[], "LUGGAGE_TYPE_PARAMS": [], 'TICKET_TYPE_PARAMS': []}
tables = {}
# Read subtables and get data from them
for row in collected_information_schema:

    #get variables
    table_schema = row['TABLE_SCHEMA']

    table_name = row['TABLE_NAME']
    table_fullname = table_schema + "." + table_name
    column_name = row['COLUMN_NAME']
    
    if table_schema == "DIM":
        read_table_to_df = (spark.read
                .format("jdbc")
                .option("url", f"jdbc:sqlserver://{prod_server_name}:{prod_port};database={prod_database_name}")
                .option("user", prod_user_name)
                .option("password", prod_password)
                .option("dbtable", table_fullname)
                .load())
        
        #saving tables in dictionary of tables
        tables[table_name] = read_table_to_df.toPandas()
        
        gained_params = collect_attributes(read_table_to_df,column_name)
        parameters[table_name + "_PARAMS"].append(gained_params)
#Dictionary of parameters        
parameters = {key: value[0] if value else [] for key, value in parameters.items()}
#print(parameters)
print(tables["DESTINATIONS"])

    DESTINATION_ID                             DESTINATION_NAME  \
0                1                            Frankfurt Airport   
1                2                             Heathrow Airport   
2                3                    Charles de Gaulle Airport   
3                4                   Amsterdam Airport Schiphol   
4                5                       Milan Malpensa Airport   
5                6                    Barcelona–El Prat Airport   
6                7          Leonardo da Vinci–Fiumicino Airport   
7                8                       Madrid Barajas Airport   
8                9                               Munich Airport   
9               10                 Vienna International Airport   
10              11                 Athens International Airport   
11              12                       Lisbon Portela Airport   
12              13                               Zurich Airport   
13              14                           Copenhagen Airpor

In [0]:
#Weighted list of nationalities
"""
weighted_nationalities = ['Czech'] * 10  # 50% Czech passengers
weighted_nationalities += ['Slovak'] * 4  # 20% Slovak passengers
weighted_nationalities += ['German'] * 2  # 10% German passengers
weighted_nationalities += ['Pole'] * 1  # 5% Pole passengers
weighted_nationalities += ['Austrian'] * 1  # 5% Austrian passengers
weighted_nationalities += ['British'] * 1  # 5% British passengers
weighted_nationalities += ['Other'] * 1  # 5% passengers from other nationalities
"""
weighted_nationalities = {"Czech": 8,
                          "Slovak": 4,
                          "German": 2,
                          "Pole": 1,
                          "Austrian": 1,
                          "British": 1,
                          "Italian":1,
                          "Other": 2}


nat_values, nat_weights = zip(*weighted_nationalities.items())

# Use random.choices() to make a random selection based on the provided weights





In [0]:
airports_dict = {    
    1: ['Lufthansa', 'Eurowings', 'Ryanair'],
    2: ['British Airways', 'EasyJet', 'Ryanair'],
    3: ['Air France', 'EasyJet', 'Ryanair'],
    4: ['KLM Royal Dutch Airlines', 'EasyJet', 'Ryanair'],
    5: ['Alitalia', 'Ryanair', 'Vueling'],
    6: ['Iberia', 'Vueling', 'Ryanair'],
    7: ['Alitalia', 'Wizz Air', 'Vueling'],
    8: ['Iberia', 'Vueling', 'Ryanair'],
    9: ['Lufthansa', 'Eurowings'],
    10: ['Ryanair', 'Wizz Air', 'Eurowings'],
    11: ['Smartwings', 'Wizz Air', 'Ryanair'],
    12: ['Smartwings', 'Ryanair', 'EasyJet'],
    13: ['Swiss International Air Lines', 'EasyJet', 'Wizz Air'],
    14: ['Scandinavian Airlines (SAS)', 'Smartwings', 'Wizz Air'],
    15: ['Scandinavian Airlines (SAS)', 'Ryanair'],
    16: ['Aer Lingus', 'Ryanair'],
    17: ['LOT Polish Airlines', 'Wizz Air'],
    18: ['Scandinavian Airlines (SAS)', 'Wizz Air', 'Smartwings'],
    19: ['LOT Polish Airlines', 'Wizz Air', 'Ryanair'],
    20: ['KLM Royal Dutch Airlines', 'Ryanair', 'Wizz Air'],
    21: ['Wizz Air', 'Ryanair', 'EasyJet'],
    22: ['Smartwings', 'Swiss International Air Lines', 'EasyJet'],
    23: ['Lufthansa', 'Smartwings'],
    24: ['LOT Polish Airlines', 'Smartwings', 'Wizz Air'],
    25: ['EasyJet', 'Vueling'],
    26: ['Iberia', 'Vueling', 'Smartwings'],
    27: ['British Airways', 'Ryanair'],
    28: ['Air France', 'EasyJet'],
    29: ['British Airways', 'EasyJet', 'Ryanair'],
    30: ['LOT Polish Airlines', 'Wizz Air'],
}


In [0]:
# df creation
flights_data = []
passengers_data = []


generated_ids = set()

def createFlight(workingDate: datetime, weatherConditions: int):
    global flights_data
    global passengers_data

    #price adjusting coefficients
    gate_price_coef = 1
    luggage_coef = 1
    ticket_coef = 1
    airline_coef = 1

    gate_revenue_price = 1 
    #delay adjusting coefficients
    gate_delay_coef = 1 
    delayAddonKoef = 0

    #occupancy coefficients
    monthOccupancyKoef = 1 
    occupancyDayTimeKoef = 1
    city_occupancy_coef = 1


    city_weights = [10, 10, 10, 10, 9, 9, 9, 9, 8, 8, 7, 7, 6, 6, 6, 6, 5, 5, 5, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3]
    #choosing random flight parameters
    city = random.choices(parameters["DESTINATIONS_PARAMS"], weights=city_weights, k=1)[0]
    airline = random.choice(airports_dict[city])
    airline_id = tables["AIRLINES"].loc[tables["AIRLINES"]["AIRLINE_NAME"] == airline, "AIRLINE_ID"].values[0]
    aircraft = random.choice(parameters["PLANES_PARAMS"])
    gate = random.choice(parameters["GATES_PARAMS"])
    plane_capacity = tables["PLANES"].loc[tables["PLANES"]["PLANE_ID"] == aircraft, "PLANE_CAPACITY"].values[0]
    
    
    if tables["GATES"].loc[tables["GATES"]["GATE_ID"] == gate, "GATE_TYPE"].values[0] == "Jet Bridge Gate":
        gate_price_coef = 1.2
        gate_revenue_price = 50000
    else:
        gate_delay_coef = 1.2
        gate_revenue_price = 30000
    

    while True:
        unique_id = random.randint(100000, 999999)
        flightGuid = f"XXX{unique_id}"

        if flightGuid not in generated_ids:
            generated_ids.add(flightGuid)
            break

    hours = random.gauss(12, 5)
    hours = max(0, min(hours, 23))
    leavingat = datetime.time(int(hours), random.randint(0, 59), 0)

    if workingDate.month in (6, 7, 8):
        monthOccupancyKoef = 1
    else:
        monthOccupancyKoef = 0.9
    
    if city <10:
        city_occupancy_coef = 1
    elif city < 20:
        city_occupancy_coef = 0.9
    else:
        city_occupancy_coef = 0.8
    
    if   0 <= hours < 7:
        occupancyDayTimeKoef = 0.85
    elif 7 <= hours < 12:
        occupancyDayTimeKoef = 1.0
    elif 12 <= hours < 17:
        occupancyDayTimeKoef = 0.95
    elif 17 <= hours < 22:
        occupancyDayTimeKoef = 1.0
    elif 22 <= hours <= 24:
        occupancyDayTimeKoef = 0.8

    totalOccupancy = 0.95 * monthOccupancyKoef * occupancyDayTimeKoef * city_occupancy_coef

    # print(f"Total Occupancy: {totalOccupancy}")

    if totalOccupancy > 1: totalOccupancy = 1


    passengers = int(random.gauss(plane_capacity * totalOccupancy, plane_capacity * 0.15))
    if passengers < 0 : passengers = 0
    if passengers > plane_capacity : passengers = plane_capacity

  
    
    ticket_price = random.gauss(2000,500)
    if ticket_price < 200:
        ticket_price = 200

    mean_age = 35
    age_std_dev = 15

    for _ in range(passengers):
        age = int(random.gauss(mean_age, age_std_dev))
        age = max(2, min(age, 90)) 
        
        first_name = fake.first_name()
        last_name = fake.last_name()
        age = int(random.gauss(mean_age, age_std_dev))
        age = max(1, min(age, 90)) 
        if airline_id > 10:
            ticket_type = 3
            airline_coef = 0.7
        
        else:
            ticket_weights = [0.1, 0.25, 0.65]
            ticket_list = parameters["TICKET_TYPE_PARAMS"]
            ticket_type = random.choices(ticket_list, ticket_weights, k=1)[0]
        luggage_weights = [0.3, 0.4, 0.05, 0.25]
        luggage_list = parameters["LUGGAGE_TYPE_PARAMS"]
        luggage_type = random.choices(luggage_list, weights=luggage_weights, k=1)[0]
        

        if luggage_type == 2:
            luggage_coef = 1.3
        elif luggage_type == 3:
            luggage_coef = 1.6
        elif luggage_type == 4:
            luggage_coef = 2.0

        if ticket_type == 2:
            ticket_coef = 3
        elif ticket_type == 1:
            ticket_coef = 6
        else:
            ticket_coef = 1

        adjusted_price = int(ticket_price * luggage_coef * ticket_coef * airline_coef * gate_price_coef)
        random_nationality = random.choices(nat_values, nat_weights)[0]
        sys_pass = datetime.datetime.now()
        gender_list = ["Male", "Female"]
        gender = random.choice(gender_list)
        new_passenger = {
            "FIRST_NAME": first_name,
            "LAST_NAME": last_name,
            "AGE": age,
            "GENDER": gender,
            "TICKET_PRICE": adjusted_price,
            "TICKET_TYPE": ticket_type,
            "LUGGAGE_TYPE": luggage_type,
            "NATIONALITY": random_nationality,
            "FLIGHT_NUMBER": flightGuid,
            "SYS_INSERTED_DT": sys_pass
        }

        passengers_data.append(new_passenger)

    #weather involvement in delays    
    weather_conditions = weatherConditions
    weather_delay_coef = 1
    if weather_conditions == 1:
        weather_delay_coef = 1
    elif weather_conditions == 2:
        weather_delay_coef = 1.1
    elif weather_conditions == 3:
        weather_delay_coef = 1.4
    elif weather_conditions == 4:
        weather_delay_coef = 1.6
    else:
        weather_delay_coef = 2

    # set delay (also base on occupancy)
    if   0.9 <= totalOccupancy <= 1.0:
        delayAddonKoef = 0.2
    elif 0.8 <= hours < 0.9:
        delayAddonKoef = 0.1
    elif 0.6 <= hours < 0.8:
        delayAddonKoef = 0.05
    else:
        delayAddonKoef = 0

    delayhours = random.gauss(0, 0.5) * gate_delay_coef * weather_delay_coef + delayAddonKoef
    delayhours = max(0, delayhours)
    deltadelay = datetime.timedelta(hours=delayhours)
    #deltadelayduration = (datetime.datetime.min + deltadelay).time()
    #delay = datetime.time(deltadelayduration.hour, deltadelayduration.minute, 0)
    delay_in_minutes = int(deltadelay.total_seconds() // 60)
    sys_fl = datetime.datetime.now()

    revenue = gate_revenue_price + (passengers * 100) 

    new_flight = {
        "DATE_OF_FLIGHT": workingDate,
        "FLIGHT_NUMBER": flightGuid,
        "DEPARTURE_TIME": leavingat,
        "DELAY_MINUTES": delay_in_minutes,
        "PASSENGERS": passengers,
        "REVENUE": revenue,
        "GATE_ID": gate,
        "DESTINATION_ID": city,
        "PLANE_ID": aircraft,
        "AIRLINE_ID": airline_id,
        "WEATHER_CONDITION_ID": weather_conditions,
        "SYS_INSERTED_DT": sys_fl
    }

    flights_data.append(new_flight)


In [0]:
UserFlights = int(User_input/27500)

In [0]:
# Winter (December, January, February)
winter_conditions = {
    1: 0.3,    # Clear
    2: 0.25,    # Cloudy
    3: 0.1,    # Rainy
    4: 0.25,   # Snowy
    5: 0.1    # Stormy
}

# Spring (March, April, May)
spring_conditions = {
    1: 0.35,   # Clear
    2: 0.35,    # Cloudy
    3: 0.25,    # Rainy
    5: 0.05     # Stormy (Assuming less frequent in spring)
}

# Summer (June, July, August)
summer_conditions = {
    1: 0.4,    # Clear
    2: 0.3,    # Cloudy
    3: 0.2,    # Rainy
    5: 0.1    # Stormy (Assuming occasional storms in summer)
}

# Autumn (September, October, November)
autumn_conditions = {
    1: 0.35,   # Clear
    2: 0.35,    # Cloudy
    3: 0.25,    # Rainy
    5: 0.05    # Stormy (Assuming less frequent in autumn)
}

# Combine the weather conditions and weights for each season
winter_values, winter_weights = zip(*winter_conditions.items())
spring_values, spring_weights = zip(*spring_conditions.items())
summer_values, summer_weights = zip(*summer_conditions.items())
autumn_values, autumn_weights = zip(*autumn_conditions.items())

In [0]:


initialDate = datetime.date(2023,1,1)
finalDate = datetime.date(2023,12,31)
summer_flights_ratio = int(UserFlights * 1.2)
december_flights_ratio = UserFlights
normal_flights_ratio = int(UserFlights * 0.8)
summer_fligts_min = int(UserFlights * 0.6)
december_flights_min = int(UserFlights * 0.5)
normal_flights_min = int(UserFlights * 0.4)

FlightsMin = 10
#generating flights between initial and final date
workingDate = initialDate
while workingDate <= finalDate:
    print (str(workingDate) + " at " + str(datetime.datetime.now()))
    #Summer
    if workingDate.month in (6,7,8):
        weather_con = random.choices(summer_values, summer_weights)[0]
        flightsPerDayRandomAddon = summer_flights_ratio
        FlightsMin = summer_fligts_min
    #Spring        
    elif workingDate.month in (3, 4, 5):
        weather_con = random.choices(spring_values, spring_weights)[0]
        flightsPerDayRandomAddon = normal_flights_ratio
        FlightsMin = normal_flights_min
    #Autumn        
    elif workingDate.month in (9, 10, 11):
        weather_con = random.choices(autumn_values, autumn_weights)[0]
        flightsPerDayRandomAddon = normal_flights_ratio
        FlightsMin = normal_flights_min

    #Winter
    elif workingDate.month in (12, 1, 2):
        weather_con = random.choices(winter_values, winter_weights)[0]
        if workingDate.month == 12:
            flightsPerDayRandomAddon = december_flights_ratio 
            FlightsMin = december_flights_min
        else:    
            flightsPerDayRandomAddon = normal_flights_ratio
            FlightsMin = normal_flights_min

    
    flightsPerDay = random.randint(FlightsMin, flightsPerDayRandomAddon) 
    i = 1

    while i < flightsPerDay:
        #here we create each flight
        createFlight(workingDate, weather_con)
        i += 1
        #end of flight per day

    #end of the major cycle
    workingDate = workingDate + datetime.timedelta(days=1)


passengers_df = pd.DataFrame(passengers_data, columns=["FIRST_NAME", "LAST_NAME", "AGE", "GENDER", "TICKET_PRICE", "TICKET_TYPE", "LUGGAGE_TYPE", "NATIONALITY", "FLIGHT_NUMBER", "SYS_INSERTED_DT"])
flights = pd.DataFrame(flights_data, columns=["DATE_OF_FLIGHT", "FLIGHT_NUMBER", "DEPARTURE_TIME", "DELAY_MINUTES", "PASSENGERS", "REVENUE", "GATE_ID", "DESTINATION_ID", "PLANE_ID", "AIRLINE_ID", "WEATHER_CONDITION_ID", "SYS_INSERTED_DT"])


#print(flights)
passengers_df.index = pd.RangeIndex(start=1, stop=len(passengers_df) + 1)
#Fact tables insert
passengers_df.to_sql('PASSENGERS', con=prod_engine, if_exists="replace", index=True, schema="FACT", index_label="ID")
flights.to_sql('DEPARTURES', con=prod_engine, if_exists="replace", index=False, schema="FACT")
#Stage tables insert
passengers_df.to_sql('PASSENGERS', con=prod_engine, if_exists="replace", index=True, schema="STAGE", index_label="ID")
flights.to_sql('DEPARTURES', con=prod_engine, if_exists="replace", index=False, schema="STAGE")


2023-01-01 at 2023-07-25 10:08:47.612085
2023-01-02 at 2023-07-25 10:08:48.384910
2023-01-03 at 2023-07-25 10:08:49.145895
2023-01-04 at 2023-07-25 10:08:50.401488
2023-01-05 at 2023-07-25 10:08:51.764394
2023-01-06 at 2023-07-25 10:08:52.903406
2023-01-07 at 2023-07-25 10:08:54.007944
2023-01-08 at 2023-07-25 10:08:55.191234
2023-01-09 at 2023-07-25 10:08:56.458384
2023-01-10 at 2023-07-25 10:08:57.872506
2023-01-11 at 2023-07-25 10:08:58.916858
2023-01-12 at 2023-07-25 10:09:00.576327
2023-01-13 at 2023-07-25 10:09:02.052375
2023-01-14 at 2023-07-25 10:09:03.531173
2023-01-15 at 2023-07-25 10:09:04.538542
2023-01-16 at 2023-07-25 10:09:05.806512
2023-01-17 at 2023-07-25 10:09:06.781696
2023-01-18 at 2023-07-25 10:09:08.210457
2023-01-19 at 2023-07-25 10:09:09.226549
2023-01-20 at 2023-07-25 10:09:10.389498
2023-01-21 at 2023-07-25 10:09:11.446247
2023-01-22 at 2023-07-25 10:09:12.339770
2023-01-23 at 2023-07-25 10:09:13.513610
2023-01-24 at 2023-07-25 10:09:14.290889
2023-01-25 at 20